In [1]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import *
from snowflake.snowpark.functions import col, lit
import getpass
import pandas as pd
import json
from get_secret import get_secret

In [2]:
connection_parameters = json.loads(get_secret())
connection_parameters.update({'warehouse':'COMPUTE_WH', 'database':'ANALYTICS_PROD','schema': 'IOATAWARE',"loglevel":'DEBUG'})
session = Session.builder.configs(connection_parameters).create()

In [3]:
print(f"Current Database and schema: {session.get_fully_qualified_current_schema()}")
print(f"Current Warehouse: {session.get_current_warehouse()}")

Current Database and schema: "ANALYTICS_PROD"."IOATAWARE"
Current Warehouse: "COMPUTE_WH"


# Import and clean snowflake views

In [4]:
view_name = "VW_AIF_REPORT_DETAIL"
flight_date = "2023/06/03"
flight_id = 111554

In [5]:
snowpark_df = session.table(view_name)
snowflake_pd_df = snowpark_df.filter((snowpark_df['Flight Date'] == flight_date) & (snowpark_df["Flight ID"] == flight_id)).to_pandas()
#snowflake_pd_df = snowflake_pd_df[snowflake_pd_df['Flight Date'] == flight_date & snowflake_pd_df["Flight ID"] == flight_id]

In [6]:
snowflake_pd_df['combined_timestamp'] = pd.to_datetime(snowflake_pd_df['Flight Date'] + ' ' + snowflake_pd_df['Flight time'])
snowflake_pd_df['combined_timestamp_mt'] = snowflake_pd_df['combined_timestamp'].dt.tz_localize('UTC').dt.tz_convert('US/Mountain')
snowflake_pd_df['Flight Date'] = snowflake_pd_df['combined_timestamp_mt'].dt.date

In [8]:
snowflake_pd_df.dtypes

Flight Date                                   object
Flight time                                   object
Flight ID                                      int32
Departure                                     object
Arrival                                       object
Reservation Number                             int32
Leg Number                                      int8
PNR                                           object
Check In Boarded                              object
Status                                        object
First Name                                    object
Last Name                                     object
Charge Type                                   object
Net Charge                                    object
Leg Start                                     object
Leg End                                       object
Charge Date                                   object
combined_timestamp                    datetime64[ns]
combined_timestamp_mt    datetime64[ns, US/Mou

In [9]:
#Remove the column we are not testing for now

In [10]:
columns_to_remove = ['First Name','Flight ID', 'Last Name','Charge Date', 'combined_timestamp','combined_timestamp_mt','Flight time']
snowflake_pd_df = snowflake_pd_df.drop(columns=columns_to_remove)
snowflake_pd_df['Flight Date'] = pd.to_datetime(snowflake_pd_df['Flight Date'])
snowflake_pd_df['Reservation Number'] = snowflake_pd_df['Reservation Number'].astype(int)
snowflake_pd_df['Leg Number'] = snowflake_pd_df['Leg Number'].astype(int)
snowflake_pd_df['Net Charge'] = snowflake_pd_df['Net Charge'].astype(float)

In [11]:
snowflake_pd_df

,Flight Date,Departure,Arrival,Reservation Number,Leg Number,PNR,Check In Boarded,Status,Charge Type,Net Charge,Leg Start,Leg End
0,2023-06-03,YYC,YYZ,7563605,2,GC6G88,B,C,Airport Improvement Fee,35.0,YYC,YYZ
1,2023-06-03,YYC,YYZ,4677091,1,YYDF7D,B,C,Airport Improvement Fee,35.0,YYC,YYZ
2,2023-06-03,YYC,YYZ,7484763,1,8XANCK,B,C,Airport Improvement Fee,35.0,YYC,YYZ
3,2023-06-03,YYC,YYZ,7550127,2,TNPX9H,B,C,Airport Improvement Fee,35.0,YYC,YYZ
4,2023-06-03,YYC,YYZ,7773831,1,JXRG4H,B,C,Airport Improvement Fee,35.0,YYC,YYZ
...,...,...,...,...,...,...,...,...,...,...,...,...
150,2023-06-03,YYC,YYZ,5141509,2,Z7Z762,B,C,Airport Improvement Fee,-35.0,YYC,YYZ
151,2023-06-03,YYC,YYZ,7758343,1,AY4PPA,B,C,Airport Improvement Fee,35.0,YYC,YYZ
152,2023-06-03,YYC,YYZ,7673592,1,4CVZHP,B,C,Airport Improvement Fee,35.0,YYC,YYZ
153,2023-06-03,YYC,YYZ,7607335,2,35EPCX,B,C,Airport Improvement Fee,35.0,YYC,YYZ


In [12]:
snowflake_pd_df.dtypes

Flight Date           datetime64[ns]
Departure                     object
Arrival                       object
Reservation Number             int32
Leg Number                     int32
PNR                           object
Check In Boarded              object
Status                        object
Charge Type                   object
Net Charge                   float64
Leg Start                     object
Leg End                       object
dtype: object

# Import and clean report data

In [16]:
report_df = pd.read_csv('AIF_REPORT_DETAIL/AIF_Report_Detail_111554.csv', skiprows=3)   

In [17]:
report_df.columns 

Index(['FlightDate1', 'Departure1', 'Arrival1', 'lng_Reservation_Nmbr',
       'lng_Leg_Nmbr', 'Reference', 'str_Res_CheckIn_Boarded',
       'str_Res_Status', 'str_First_Name', 'str_Last_Name',
       'str_GL_Charge_Type_Desc', 'NetCharge', 'LegDeparture', 'LegArrival',
       'ChargeDate'],
      dtype='object')

In [18]:
report_df.dtypes

FlightDate1                object
Departure1                 object
Arrival1                   object
lng_Reservation_Nmbr        int64
lng_Leg_Nmbr                int64
Reference                  object
str_Res_CheckIn_Boarded    object
str_Res_Status             object
str_First_Name             object
str_Last_Name              object
str_GL_Charge_Type_Desc    object
NetCharge                  object
LegDeparture               object
LegArrival                 object
ChargeDate                 object
dtype: object

In [19]:
report_df

,FlightDate1,Departure1,Arrival1,lng_Reservation_Nmbr,lng_Leg_Nmbr,Reference,str_Res_CheckIn_Boarded,str_Res_Status,str_First_Name,str_Last_Name,str_GL_Charge_Type_Desc,NetCharge,LegDeparture,LegArrival,ChargeDate
0,2023/06/03,YYC,YYZ,5166995,2,XVXP78,B,C,MATT GREGORY,VISITACION,Airport Improvement Fee,$35.00,YYC,YYZ,11/29/2022
1,2023/06/03,YYC,YYZ,5401723,1,2SDFYX,B,C,CHUNGYOUNG,LIM,Airport Improvement Fee,$35.00,YYC,YYZ,12/29/2022
2,2023/06/03,YYC,YYZ,5397156,2,3TKS25,B,C,UTKARSH,SHARMA,Airport Improvement Fee,$35.00,YYC,YYZ,12/28/2022
3,2023/06/03,YYC,YYZ,5401723,1,2SDFYX,B,C,SEUNGYEOB,LIM,Airport Improvement Fee,$35.00,YYC,YYZ,12/29/2022
4,2023/06/03,YYC,YYZ,5401723,1,2SDFYX,B,C,HEESOOK,KIM,Airport Improvement Fee,$35.00,YYC,YYZ,12/29/2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,2023/06/03,YYC,YYZ,7550127,2,TNPX9H,B,C,YAN KWONG IAN,CHAN,Airport Improvement Fee,$35.00,YYC,YYZ,5/16/2023
151,2023/06/03,YYC,YYZ,7550127,2,TNPX9H,B,C,SIU HONG,SUEN,Airport Improvement Fee,$35.00,YYC,YYZ,5/16/2023
152,2023/06/03,YYC,YYZ,7563605,2,GC6G88,B,C,MANOJ,CHATTERJEE,Airport Improvement Fee,$35.00,YYC,YYZ,5/17/2023
153,2023/06/03,YYC,YYZ,7563605,2,GC6G88,B,C,SHYAMALI,CHATTERJEE,Airport Improvement Fee,$35.00,YYC,YYZ,5/17/2023


In [20]:
columns_to_remove = ['str_First_Name', 'str_Last_Name','ChargeDate']
report_df = report_df.drop(columns=columns_to_remove)

report_df['FlightDate1'] = pd.to_datetime(report_df['FlightDate1'])
report_df['lng_Reservation_Nmbr'] = report_df['lng_Reservation_Nmbr'].astype(int)
report_df['lng_Leg_Nmbr'] = report_df['lng_Leg_Nmbr'].astype(int)
report_df['NetCharge'] = report_df['NetCharge'].str.replace('$', '')
report_df['NetCharge'] = report_df['NetCharge'].str.replace(')', '')
report_df['NetCharge'] = report_df['NetCharge'].str.replace('(', '-')
report_df['NetCharge'] = report_df['NetCharge'].astype(float)

# Compare two dataframes

In [21]:
snowflake_pd_df =snowflake_pd_df.drop_duplicates()
#snowflake_pd_df

In [22]:
report_df = report_df.drop_duplicates()
#report_df

In [23]:
snowflake_pd_df.columns = report_df.columns

In [24]:
merged = snowflake_pd_df.merge(report_df, how = 'outer', indicator = True)
snowflake_only = merged[merged['_merge'] == "left_only"]
common = merged[merged['_merge'] == "both"]
report_only = merged[merged['_merge'] == "right_only"]

In [25]:
#common

In [26]:
def calculate_df2_accuracy(df1, df2):
    merged = df1.merge(df2, how = 'outer', indicator = True)
    #matched = merged[merged['_merge'] == "both"]
    df1_only = merged[merged['_merge'] == "left_only"]
    df2_only = merged[merged['_merge'] == "right_only"]

    # Calculate the total number of elements
    total_elements = len(df2)

    # Calculate accuracy as a percentage
    accuracy = ((total_elements - len(df2_only)) / total_elements) * 100

    return accuracy


In [27]:
accuracy_percentage = calculate_df2_accuracy(report_df,snowflake_pd_df)
print(f"Accuracy: {accuracy_percentage:.2f}%" " snowflake records in report\nThe number of records in snowflake but not in report is", len(snowflake_only))

Accuracy: 100.00% snowflake records in report
The number of records in snowflake but not in report is 0


In [28]:
accuracy_percentage = calculate_df2_accuracy(snowflake_pd_df, report_df)
print(f"Accuracy: {accuracy_percentage:.2f}%"" report records in snowflake\nThe number of records in report but not in snowflake is", len(report_only))

Accuracy: 100.00% report records in snowflake
The number of records in report but not in snowflake is 0


# Output the difference file

In [29]:
try:
    snowflake_only.to_csv("snowflake_only_record.csv", index=False) 
except:
    print("No snowflake only records")
try:
    report_only.to_csv("report_only_record.csv", index=False) 
except:
    print("No report only records")